# Import libraries and define util functions

In [2]:

import pandas as pd
from bs4 import BeautifulSoup

from llama_index import VectorStoreIndex
from llama_index.node_parser import UnstructuredElementNodeParser
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

from llama_index.response_synthesizers import (
    get_response_synthesizer
)
import os

In [ ]:
# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = ''

In [3]:
# Path to file
path = 'ucl.html'

In [4]:
def extract_tables(path, stop_header_text):
    with open(path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    
    tables = []
    stop_header_found = False

    for element in soup.find_all(['h2', 'table']):
        if element.name == 'h2' and element.get_text() == stop_header_text:
            stop_header_found = True
        elif element.name == 'table' and not stop_header_found:
            tables.append(element)

    return tables

In [5]:
def get_table_shape(table):
    rows = table.find_all("tr")
    row_count = len(rows)
    col_count = 0
    for row in rows:
        cols = row.find_all(["th", "td"])
        col_count_temp = len(cols)
        if col_count_temp > col_count:
            col_count = col_count_temp
    return (row_count, col_count)

In [6]:
def create_none_matrix(shape):
    rows, columns = shape
    return [[None] * columns for _ in range(rows)]

In [7]:
def get_ref_node_of_table(table_node, nodes):
    id_n = table_node.node_id
    for n in nodes:
        if n.id_ == str(id_n + "_ref"):
            return n

In [8]:
def html_to_df(table):
    table_shape = get_table_shape(table)
    none_table = create_none_matrix(table_shape)
    skip_next = None
    cell_rowspan = None
    marked_coords = []
    for i, row in enumerate(table.find_all("tr")):
        cells_of_row = row.find_all(["th", "td"])

        if (len(cells_of_row) == 1):
            if cells_of_row[0].get("colspan"):
                if int(cells_of_row[0].get("colspan")) == int(table_shape[1]):
                    continue
            
        for j in range(table_shape[1]):
            if skip_next:
                skip_next = False
                continue

            if none_table[i][j] is None:
                try:
                    cell = cells_of_row[0]
                except:
                    none_table[i][j] = ''
                    continue
                cell_rowspan = cell.get("rowspan")
                cell_colspan = cell.get("colspan")
                cell_text = cell.get_text().replace("\n", "")

                #if not marked_coords:
                none_table[i][j] = cell_text
                if cell_rowspan:
                    for x in range(i, int(i) + int(cell_rowspan)):
                        none_table[x][j] = cell_text
                    marked_coords.append((i,j,cell_rowspan))
                    #del cells_of_row[0]
                if cell_colspan:
                    for y in range(j+1, (int(j+1) + int(cell_colspan)-1)):
                        none_table[i][y] = 'ALL' if cell_text not in ['\n', ''] else ''
                    marked_coords.append((i,j,cell_colspan))
                    skip_next = True
                
                del cells_of_row[0]

    none_table = [row for row in none_table if any(cell is not None for cell in row)]

    df = pd.DataFrame(none_table[1:], columns=none_table[0])
    return df

# EmbeddedTablesUnstructuredRetrieverPack (under the hood)

## Read the file and parse it into nodes with UnstructuredElementNodeParser

In [9]:
reader = FlatReader()

docs = reader.load_data(Path(path))

In [10]:
node_parser = UnstructuredElementNodeParser()

In [11]:
raw_nodes = node_parser.get_nodes_from_documents(docs)

100%|██████████| 13/13 [01:51<00:00,  8.56s/it]


In [13]:
len(raw_nodes)

51

## Proceed without updating the tables first, to have benchmark responses for comparison

In [13]:
base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(
            raw_nodes
        )

In [14]:
vector_index = VectorStoreIndex(nodes=base_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=4)

In [15]:
recursive_retriever = RecursiveRetriever(
            "vector",
            retriever_dict={"vector": vector_retriever},
            node_dict=node_mappings,
            verbose=False,
        )

In [16]:
query_engine = RetrieverQueryEngine.from_args(retriever=recursive_retriever)

In [17]:
print(query_engine.query("how many number of times has casemiro won champions league").response)
print(query_engine.query("how many number of times has luka modric won champions league").response)
print(query_engine.query("how many number of times has juan santisteban won champions league").response)
print(query_engine.query("how many number of times has pique won champions league").response)
print(query_engine.query("how many number of times has varane won champions league").response)

Casemiro has won the Champions League 4 times.
Luka Modrić has won the UEFA Champions League once.
Juan Santisteban has not won the Champions League.
Gerard Piqué has won the Champions League 3 times.
Varane has won the Champions League 4 times.


**Correct answers were;**\
Casemiro : 5 times\
Modric : 5 times\
Santisteban : 4 times\
Pique : 4 times\
Varane : 4 times

In [18]:
print(query_engine.query("how many appearances does zlatan ibrahimovic have in the champions league").response)
print(query_engine.query("how many appearances does paul scholes have in the champions league").response)
print(query_engine.query("how many appearances does philipp lahm have in the champions league").response)
print(query_engine.query("how many appearances does dani alves have in the champions leaguee").response)
print(query_engine.query("how many appearances does thiago silva have in the champions league").response)
print(query_engine.query("Who are the top 5 players that have the most appearances in the champions league").response)

Zlatan Ibrahimović has made 116 appearances in the UEFA Champions League.
Paul Scholes has 124 appearances in the UEFA Champions League.
Philipp Lahm has 105 appearances in the UEFA Champions League.
Dani Alves has made 111 appearances in the UEFA Champions League.
Thiago Silva has 105 appearances in the UEFA Champions League.
The top 5 players with the most appearances in the UEFA Champions League are Cristiano Ronaldo, Iker Casillas, Lionel Messi, Karim Benzema, and Xavi.


**Correct answers were;**\
Ibrahimovic : 124 times\
Scholes : 124 times\
Lahm : 112 times\
Dani Alves : 111 times\
Thiago Silva : 105 times\
Top 5 Players: C. Ronaldo, Casillas, Messi, Benzema, Xavi

In [19]:
# NOTE: The queries below originally didn't include the "look for the answer in" part but retriever failed to find the correct node. 
# Since the scope of this work is to improve the quality of information retrieved WITHIN the node, 
# the process of finding the correct node is by-passed, so to speak, with prompt engineering 
print(query_engine.query("when was malmö ff the runner-up? look for the answer in the table with columns club, title(s), runners-up, seasons won, seasons runner-up").response)
print(query_engine.query("when was tottenham the runner-up? look for the answer in the table with columns club, title(s), runners-up, seasons won, seasons runner-up").response)
print(query_engine.query("what are the names of the clubs that ibrahimovic played for in the champions league? Look for the answer in the table under 'most appearances' section").response)
print(query_engine.query("In what years did Luis Figo play in the champions league? Look for the answer in the table under the 'most appearances' section").response)

Malmö FF was the runner-up in the UEFA Champions League in the season 1978–79.
Tottenham Hotspur was the runner-up in the UEFA Champions League once.
Ajax, Juventus, Inter Milan, Barcelona, Milan, Paris Saint-Germain, Manchester United, Valencia, Fiorentina, Eintracht Frankfurt, Partizan, Panathinaikos, Leeds United, Saint-Étienne, Borussia Mönchengladbach, Club Brugge, Malmö FF, Roma, Sampdoria, Bayer Leverkusen, Monaco, Arsenal, and Tottenham Hotspur.
Luis Figo played in the UEFA Champions League in the following years: 1997, 1998, 2009.


**Correct answers were;**\
Malmö was the runner-up in 1978-79\
Tottenham was the runner-up in 2018-2019\
Ibrahimovic played for Ajax, Juventus, Inter Milan, Barcelona, Milan, Paris Saint-Germain, Manchester United in the UCL\
Figo played in the UCL from 1997 to 2009

## Updating the tables and generating responses this way

The unstructured node parser takes a `llama_index.schema.Document` as input that is read with a LlamaIndex `FlatReader`. Even though this Document contains most of the html tags such as `<span>, <tr>, <td>` , and the `UnstructuredElementNodeParser` does a great job detecting the tables, it fails to extract the full information under some conditions including but not limited to:
* The table has merged cells
* The cell value includes references
* The cell value is a comma seperated list

This situation causes information loss, which can be recovered considering the original data source was an HTML and actually includes all the necessary tags to capture the information of the table perfectly.

Therefore, this notebook aims to follow the pipeline of the `EmbeddedTablesUnstructuredRetrieverPack` but just change the content of the `TextNode` objects that contain the tables with a custom function

### Print raw table nodes, update the nodes and print the updated nodes

In [20]:
# The function below returns the HTML content of the tables up to the section specified in the string. 
# In this case, we are only concerned with the tables that come before the References section because there many tables at the bottom of the wiki page that we are not interested in.
tables = extract_tables(path, 'References') 

# We loop through the HTML contents of the tables and turn each of them into a pandas DataFrame first, then return the string of the DataFrames.
# This way, table contents as strings have a more organized structure
tables_corrected_texts = []
for table in tables:
    df = html_to_df(table)
    if df.shape[0] > 0: # There are two "tables" in the HTML document that are actually pictures. They are formatted as a HTML table but they shape (0, 1). This is to exclude them.
        tables_corrected_texts.append(df.to_string())

In [22]:
# Before updating the tables, we print the TextNodes that contain the table content to see the difference
c=0
for i, node in enumerate(raw_nodes):
    if str(node.id_).endswith("_table"):
        ref_node = get_ref_node_of_table(node, raw_nodes)
        print("Table " + str(c+1) + " is at node with index " + str(i))
        print("Table summary: " + str(ref_node.text))
        print(node.text)
        print("\n\n")
        c += 1
    if c > 10: # We are only interested in the first 11 tables of the HTML document as explained above
        break

Table 1 is at node with index 2
Table summary: Summary: UEFA Champions League Information
                                                                                                                                                                                                                                                                                                                                                          
0           Organising body                                                                                                                                                                                                           UEFA                                                                                                                
1                   Founded                                                                                                                                                                                             

In [23]:
# Now, we update each table with their newly created "correct texts" and print them
c=0
raw_nodes_updated = raw_nodes.copy()
for i, node in enumerate(raw_nodes_updated):
    if str(node.id_).endswith("_table"):
        ref_node = get_ref_node_of_table(node, raw_nodes_updated)
        node.text = tables_corrected_texts[c]
        print("Table " + str(c+1) + " is at node with index " + str(i) + " and has table_id:" + str(node.id_))
        print("Table summary: " + str(ref_node.text))
        print(node.text)
        print("\n\n")
        c += 1
    if c > 10:
        break

Table 1 is at node with index 2 and has table_id:id_155_table
Table summary: Summary: UEFA Champions League Information
           Organising body                                                                  UEFA
0                  Founded                          1955; 69 years ago (1955)(rebranded in 1992)
1                   Region                                                                Europe
2          Number of teams                                 32 (group stage)79, 80, or 81 (total)
3            Qualifier for            UEFA Super CupFIFA Club World CupFIFA Intercontinental Cup
4     Related competitions  UEFA Europa League (2nd tier)UEFA Europa Conference League(3rd tier)
5        Current champions                                           Manchester City (1st title)
6  Most successful club(s)                                               Real Madrid (14 titles)
7  Television broadcasters                                                  List of broadcasters
8      

### Proceed to the rest of the pipeline with the updated nodes.

In [24]:
base_nodes_upd, node_mappings_upd = node_parser.get_base_nodes_and_mappings(
            raw_nodes_updated
        )

In [25]:
vector_index_upd = VectorStoreIndex(nodes=base_nodes_upd)
vector_retriever_upd = vector_index_upd.as_retriever(similarity_top_k=4)

In [26]:
recursive_retriever_upd = RecursiveRetriever(
            "vector",
            retriever_dict={"vector": vector_retriever_upd},
            node_dict=node_mappings_upd,
            verbose=False,
        )

In [27]:
from llama_index.callbacks.base import CallbackManager
from llama_index.prompts import SelectorPromptTemplate
from llama_index.prompts.base import PromptTemplate
from llama_index.prompts.prompt_type import PromptType

# The new table texts also include the columns as the first row (as opposed to the original text), therefore we slightly change the QA PROMPT TEMPLATE
MY_TEXT_QA_PROMPT_TMPL = (
    "The context below may or may not contain tables. You can recognize the tables by the numbered new line characters such as \\n0, \\n1, etc. Each new line coded like this is a different row of the table. Widespaces between values are cell borders (different columns). \n"
    "If a table exists, the first line/row of the table is the column names. If the query includes a column name, take it into account."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

MY_TEXT_QA_PROMPT = PromptTemplate(
    MY_TEXT_QA_PROMPT_TMPL, prompt_type=PromptType.QUESTION_ANSWER
)

MY_TEXT_QA_PROMPT_SEL = SelectorPromptTemplate(
    default_template=MY_TEXT_QA_PROMPT,
)

In [28]:
resp_synth = get_response_synthesizer(
                service_context=recursive_retriever_upd.get_service_context(),
                callback_manager=CallbackManager(),
                text_qa_template=MY_TEXT_QA_PROMPT_SEL
                )

In [29]:
query_engine_upd = RetrieverQueryEngine.from_args(retriever=recursive_retriever_upd, response_synthesizer=resp_synth)

In [30]:
print(query_engine_upd.query("how many number of times has casemiro won champions league").response)
print(query_engine_upd.query("how many number of times has luka modric won champions league").response)
print(query_engine_upd.query("how many number of times has juan santisteban won champions league").response)
print(query_engine_upd.query("how many number of times has pique won champions league").response)
print(query_engine_upd.query("how many number of times has varane won champions league").response)

Casemiro has won the Champions League 5 times.
Luka Modrić has won the Champions League 5 times.
Juan Santisteban has won the Champions League 4 times.
Gerard Piqué has won the Champions League 4 times.
4


In [31]:
print(query_engine_upd.query("how many appearances does zlatan ibrahimovic have in the champions league").response)
print(query_engine_upd.query("how many appearances does paul scholes have in the champions league").response)
print(query_engine_upd.query("how many appearances does philipp lahm have in the champions league").response)
print(query_engine_upd.query("how many appearances does dani alves have in the champions leaguee").response)
print(query_engine_upd.query("how many appearances does thiago silva have in the champions league").response)
print(query_engine_upd.query("Who are the top 5 players that have the most appearances in the champions league the most?").response)

Zlatan Ibrahimović has 124 appearances in the UEFA Champions League.
Paul Scholes has 124 appearances in the UEFA Champions League.
Philipp Lahm has 112 appearances in the UEFA Champions League.
Dani Alves has 111 appearances in the UEFA Champions League.
Thiago Silva has 105 appearances in the UEFA Champions League.
1. Cristiano Ronaldo
2. Iker Casillas
3. Lionel Messi
4. Karim Benzema
5. Xavi


In [36]:
# NOTE: The queries below originally didn't include the "look for the answer in" part but retriever failed to find the correct node. 
# Since the scope of this work is to improve the quality of information retrieved WITHIN the node, 
# the process of finding the correct node is by-passed, so to speak, with prompt engineering
print(query_engine_upd.query("when was malmö ff the runner-up? look for the answer in the table with columns club, title(s), runners-up, seasons won, seasons runner-up").response)
print(query_engine_upd.query("when was tottenham the runner-up? look for the answer in the table with columns club, title(s), runners-up, seasons won, seasons runner-up").response)
print(query_engine_upd.query("what are the names of the clubs that zlatan ibrahimovic played for in the champions league? Look for the answer in the table under 'most appearances' section").response)
print(query_engine_upd.query("In what years did Luis Figo play in the champions league? Look for the answer in the table under the 'most appearances' section").response)

Malmö FF was the runner-up in the 1979 season.
Tottenham was the runner-up in the UEFA Champions League in the 2018-2019 season.
The clubs that Zlatan Ibrahimovic played for in the Champions League are Ajax, Juventus, Inter Milan, Barcelona, Milan, Paris Saint-Germain, and Manchester United.
Luis Figo played in the Champions League from 1997 to 2009.


**All answers are correct!**